In [1]:
from fastcore.all import parallel
import pandas as pd
import requests 
import bs4
import tqdm

Missing colon in file Path('/home/ptrbdr/.config/matplotlib/stylelib/texty.mplstyle'), line 14 ("                               'fc4f30', ")
Missing colon in file Path('/home/ptrbdr/.config/matplotlib/stylelib/texty.mplstyle'), line 15 ("                               'e5ae38',")
Missing colon in file Path('/home/ptrbdr/.config/matplotlib/stylelib/texty.mplstyle'), line 16 ("                               '6d904f',")
Missing colon in file Path('/home/ptrbdr/.config/matplotlib/stylelib/texty.mplstyle'), line 17 ("                               '8b8b8b',")
Missing colon in file Path('/home/ptrbdr/.config/matplotlib/stylelib/texty.mplstyle'), line 18 ("                               '810f7c'])")
Bad value in file Path('/home/ptrbdr/.config/matplotlib/stylelib/texty.mplstyle'), line 13 ("axes.prop_cycle: cycler(color=['008fd5', "): Key axes.prop_cycle: 'cycler(color=['008fd5',' is not a valid cycler construction: unexpected EOF while parsing (<string>, line 1)
Bad value in file Path('/home

First lets get a list of all permits from [water agency website](https://e-services.davr.gov.ua/site/permit-registry). 

In [5]:
# we define number of pages currently there are 1012 pages with permits
page_numbers = [a+1 for a in range(1012)]

In [27]:
# handler function for downloading pages into folder
def download_handler(idx):
    try:
        request = requests.get(f'https://e-services.davr.gov.ua/site/permit-registry?page={idx}')
        with open(f'htmls/{idx}.html', 'w') as file:
            file.write(request.text)
        return (True, idx)
    except:
        return (False, idx)
    

In [28]:
results = parallel(download_handler, page_numbers, n_workers=8)

# you may need to run those indexes second time if any problems with network
not_processed = results.filter(f=lambda x: x[0] != True).map(lambda x: x[1])

In [42]:
def parse_general_table(page):
    table = pd.read_html(f'htmls/{page}.html')[0]

    table.columns = [d[0] for d in table.columns]

    idxs = []
    for d in bs4.BeautifulSoup(open(f'htmls/{page}.html')).select('tr a'):
        if d['href'].split('=')[1] != '1&sort':
            idxs.append(d['href'].split('=')[1] )


    table['Доступні дії'] = [d for d in idxs if 'sort' not in d ]
    table['page_number_of_permit'] = page

    return table 

In [43]:
parsing_res = []
for d in tqdm.tqdm(page_numbers[:20]):
    t = parse_general_table(d)
    parsing_res.append(t)

100%|██████████| 20/20 [00:00<00:00, 31.01it/s]


In [44]:
general_table_data = pd.concat(parsing_res)
general_table_data.to_csv('overal_permits_table.csv', index=False)

We have a list of all permits, we have to select unique permits that are still active and scrape base information from each permit

In [47]:
ids_of_permits = general_table_data.loc[general_table_data['Поточний стан'] == 'діючий']['Доступні дії'].unique()

In [71]:
def parse_permit_table(page):
    try:
        response = requests.get(f'https://e-services.davr.gov.ua/parlor/p-permit-registry/view-permit?id={page}')
        with open(f'permits/{page}.html', 'w') as file:
            file.write(response.text)
        return (True, page)
    except:
        return (False, page)

    

    

In [72]:
results_permits = parallel(parse_permit_table, ids_of_permits, n_workers=8)

In [58]:
# you may need to run those indexes second time if any problems with network
not_processed = results_permits.filter(f=lambda x: x[0] != True).map(lambda x: x[1])

Download pages with the coordinates of extraction sites

In [73]:
water_extraction_place_id = pd.read_csv('water_extraction_place_id.csv')

In [76]:
lists_of_ids = water_extraction_place_id.apply(lambda x: (x['permit_id'],x['water_extraction_place_id']), axis=1).to_list()

## Send request 

Video describing where to get headers and cookies for request:
https://www.youtube.com/watch?v=DxeSGUM16_4 

In [1]:
for d in tqdm.tqdm(lists_of_ids):
    cookies = {
        '_ga': 'GA1.3.1894082937.1589469539',
        'PHPSESSID': 'c1jp23efiemhffg0rtkkr9omt2',
        '_csrf': '39f56d58225ed9b6ba51201d83d09a599e86b76b7c19a6483866374286cdfb4fa%3A2%3A%7Bi%3A0%3Bs%3A5%3A%22_csrf%22%3Bi%3A1%3Bs%3A32%3A%22s0F0qDo6U8lbh0ZyA_Alze2wyER30y3u%22%3B%7D',
        '_gid': 'GA1.3.965287914.1592209945',
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:75.0) Gecko/20100101 Firefox/75.0',
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Language': 'en-US,en;q=0.5',
        'X-CSRF-Token': 'XKny1CExvGWArojHM4p2dI7v6ihpJIK_KI26QTJkrCIvmbTkUHXTU9WW5KVbuiwNz7CrRBNBsMhRyOhyAh2fVw==',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'X-Requested-With': 'XMLHttpRequest',
        'Origin': 'https://e-services.davr.gov.ua',
        'Connection': 'keep-alive',
        'Referer': f'https://e-services.davr.gov.ua/parlor/p-permit-registry/view-permit?id={d[0]}',
        'Pragma': 'no-cache',
        'Cache-Control': 'no-cache',
    }

    data = {
      'id': d[1]
    }

    response = requests.post('https://e-services.davr.gov.ua/parlor/p-permit-registry/view-water-place',
                             headers=headers, 
                             cookies=cookies, 
                             data=data)
    
    with open(f'water_extract_locations/{d[0]}_{d[1]}.html', 'w') as file:
            file.write(response.json()['html'])